## Keras MNIST with TPUs

Here is a very quick implemention and walkthrough to show using TPUs with Keras

In [1]:
import numpy as np

import tensorflow as tf
import time
import os

import tensorflow.keras
from tensorflow.keras.datasets import mnist, fashion_mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten,Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [2]:
print(tf.__version__)
print(tf.keras.__version__)

1.13.1
2.2.4-tf


In [3]:
import tensorflow as tf

tpu_name = 'tpu-us-central1-a-00'
tpu_zone = 'us-central1-a'
tpu_project = 'pipelineai2'

tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(
            tpu=tpu_name,
            zone=tpu_zone,
            project=tpu_project)

TPU_ADDRESS = tpu_cluster_resolver.get_master()
print(TPU_ADDRESS)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

grpc://10.128.100.2:8470


### Normal MNIST Stuff

In [4]:
batch_size = 1024
num_classes = 10
epochs = 5
learning_rate = 0.001

# input image dimensions
img_rows, img_cols = 28, 28

In [5]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [6]:
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [7]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [8]:
# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

## Use tf.data

you need to make sure you have drop_remainder = True as TPUs need to have a fixed shape

In [9]:
def train_input_fn(batch_size=1024):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.cache() # Loads the data into memory since its such a small dataset
    dataset = dataset.shuffle(1000, reshuffle_each_iteration=True)
    dataset = dataset.repeat() 
    dataset = dataset.batch(batch_size, drop_remainder=True)


    # Return the dataset.
    return dataset

In [10]:
def test_input_fn(batch_size=1024):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((x_test,y_test))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.cache()
    dataset = dataset.shuffle(1000, reshuffle_each_iteration=True)
    dataset = dataset.repeat()
    dataset = dataset.batch(batch_size, drop_remainder=True)


    # Return the dataset.
    return dataset

## Make the model

you must pass in an input shape and batch size as TPUs (and XLA) require fixed shapes 

The rest of the model is just a simple CNN

In [11]:
Inp = tf.keras.Input(
      name='input', shape=input_shape, batch_size=batch_size, dtype=tf.float32)
x = Conv2D(32, kernel_size=(3, 3), activation='relu',name = 'Conv_01')(Inp)
x = MaxPooling2D(pool_size=(2, 2),name = 'MaxPool_01')(x)
x = Conv2D(64, (3, 3), activation='relu',name = 'Conv_02')(x)
x = MaxPooling2D(pool_size=(2, 2),name = 'MaxPool_02')(x)
x = Conv2D(64, (3, 3), activation='relu',name = 'Conv_03')(x)
x = Flatten(name = 'Flatten_01')(x)
x = Dense(64, activation='relu',name = 'Dense_01')(x)
x = Dropout(0.5,name = 'Dropout_02')(x)
output = Dense(num_classes, activation='softmax',name = 'Dense_02')(x)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
model = tf.keras.Model(inputs=[Inp], outputs=[output])

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (1024, 28, 28, 1)         0         
_________________________________________________________________
Conv_01 (Conv2D)             (1024, 26, 26, 32)        320       
_________________________________________________________________
MaxPool_01 (MaxPooling2D)    (1024, 13, 13, 32)        0         
_________________________________________________________________
Conv_02 (Conv2D)             (1024, 11, 11, 64)        18496     
_________________________________________________________________
MaxPool_02 (MaxPooling2D)    (1024, 5, 5, 64)          0         
_________________________________________________________________
Conv_03 (Conv2D)             (1024, 3, 3, 64)          36928     
_________________________________________________________________
Flatten_01 (Flatten)         (1024, 576)               0         
__________

In [14]:
# Use a tf optimizer rather than a Keras one for now
opt = tf.train.AdamOptimizer(learning_rate)

model.compile(
      optimizer=opt,
      loss='categorical_crossentropy',
      metrics=['acc'])

## Creating the TPU from a Keras Model

tf.contrib.tpu.keras_to_tpu_model will eventually go away and you will pass it into the model.compile as a distribution strategy, but for TensorFlow 1.x this works. 

In [ ]:
tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)))

INFO:tensorflow:Querying Tensorflow master (grpc://10.128.100.2:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 5967728145050075118)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 9776667592719210851)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 13136854972760794747)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 4162951683664012592)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 8059124576865597710)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/

In [ ]:
tpu_model.summary()

## Training using tf.data pipeline 

obviously training MNIST on a TPU is a bit overkill and the TPU barely gets a chance to warm up

In [ ]:
tpu_model.fit(
    train_input_fn,
    steps_per_epoch = 60,
    epochs=10,
)

In [ ]:
tpu_model.save_weights('./MNIST_TPU_1024.h5', overwrite=True)

In [ ]:
tpu_model.evaluate(test_input_fn,
    steps = 100)

### Converting the model back to a CPU model

In [ ]:
cpu_model = tpu_model.sync_to_cpu()